# Optional - Set Black Formatter

Loads Black formatter to automatically format notebook cells, requires nb_black to be installed. 
To install simply run `pip3 install nb_black`.

In [ ]:
%load_ext nb_black

# Imports and Config

In [ ]:
import os
import sys
import yaml

import rclpy
import numpy as np

# config file libraries
from ament_index_python.packages import get_package_share_directory
from moveit_configs_utils import MoveItConfigsBuilder

# message libraries
from geometry_msgs.msg import PoseStamped, Pose
from moveit_msgs.msg import AttachedCollisionObject, CollisionObject, PlanningScene
from shape_msgs.msg import SolidPrimitive

# moveit_py
from moveit_py.planning import MoveItPy
from moveit_py.core import RobotState

# manually create config (since we aren't starting moveit_py from launchfile)
moveit_config = (
    MoveItConfigsBuilder(
        robot_name="panda", package_name="moveit_resources_panda_moveit_config"
    )
    .robot_description(file_path="config/panda.urdf.xacro")
    .trajectory_execution(file_path="config/gripper_moveit_controllers.yaml")
    .moveit_cpp(
        file_path=get_package_share_directory("moveit_py_example")
        + "/config/moveit_py.yaml"
    )
    .to_moveit_configs()
).to_dict()


# define box that we will use throughout tutorial
box = SolidPrimitive()
box.type = SolidPrimitive.BOX
box.dimensions = [0.1, 0.1, 0.1]

# define flattened box that we will use throughout tutorial
box_flattened = SolidPrimitive()
box_flattened.type = SolidPrimitive.BOX
box_flattened.dimensions = [0.5, 0.8, 0.01]

# MoveItPy Setup

In [ ]:
rclpy.init()


# instantiate MoveItPy instance and get planning component
moveit = MoveItPy(node_name="moveit_py", config_dict=moveit_config)
panda_arm = moveit.get_planning_component("panda_arm")

# Plan and Execute with Collision Object in Scene

In [ ]:
# get planning scene monitor instance from moveit_py instance
psm = moveit.get_planning_scene_monitor()

# create collision object
co = CollisionObject()
co.operation = CollisionObject.ADD
co.id = "box_1"
co.header.frame_id = "panda_link0"
co.primitives = [box_flattened]
pose = Pose()
pose.position.x = 0.45
pose.position.y = 0.35
pose.position.z = 0.8
co.primitive_poses = [pose]

# lock the planning scene and return read_write access to a planning scene
with psm.read_write() as scene:

    # add a collision object to the scene
    scene.apply_collision_object(co)

    # check if the current robot state is in collision
    robot_state = scene.current_state
    robot_state.update()  # required to update transforms
    robot_collision_status = scene.is_state_colliding(
        robot_state=robot_state, group="panda_arm", verbose=False
    )

    # check if a user-defined robot state is in collision
    pose_goal = Pose()
    pose_goal.position.x = 0.25
    pose_goal.position.y = 0.25
    pose_goal.position.z = 0.0
    pose_goal.orientation.w = 1.0
    robot_state.set_from_ik("panda_arm", pose_goal, "panda_link8", timeout=5.0)
    robot_state.update()  # required to update transforms

    # This robot state should be in collision
    robot_collision_status = scene.is_state_colliding(
        robot_state=robot_state, group="panda_arm", verbose=False
    )

# set plan start state using predefined state
panda_arm.set_start_state("ready")

# set pose goal using predefined state
panda_arm.set_goal("extended")

# plan to goal
plan_solution = panda_arm.plan()

if plan_solution:
    panda_arm.execute()

# Plan and Execute with Attached Collision Object

In [ ]:
with psm.read_write() as scene:
    # attach collision object to robot
    a_co = AttachedCollisionObject()
    a_co.link_name = "panda_link8"
    a_co.touch_links = ["panda_link8", "panda_rightfinger", "panda_leftfinger"]

    co = CollisionObject()
    co.operation = CollisionObject.ADD
    co.id = "box_attached"
    co.header.frame_id = "panda_link0"
    co.primitives = [box]

    robot_state = scene.current_state
    robot_state.update()  # required to update transforms
    eef_pose = robot_state.get_pose("panda_link8")
    eef_pose.position.x += 0.15
    co.primitive_poses = [eef_pose]
    a_co.object = co
    scene.apply_attached_collision_object(a_co)

# set plan start state to the current state
panda_arm.set_start_state_to_current_state()

# set pose goal with PoseStamped message
pose_goal = PoseStamped()
pose_goal.header.frame_id = "panda_link0"
pose_goal.pose.orientation.w = 1.0
pose_goal.pose.position.x = 0.28
pose_goal.pose.position.y = -0.2
pose_goal.pose.position.z = 0.2
panda_arm.set_goal(goal_pose_msg=pose_goal, link_name="panda_link8")

# plan to goal
plan_solution = panda_arm.plan()


# Examining Properties of Plan Solution

In [ ]:
# Examine the plan solution
solution_trajectory = plan_solution.trajectory

print(
    "The number of waypoints in solution: {}".format(solution_trajectory.num_waypoints)
)
print("The duration of the solution: {}".format(solution_trajectory.duration))
print(
    "The average segment duration of solution: {}".format(
        solution_trajectory.average_segment_duration
    )
)

# Checking for Collisions and Path Validity on Updated Planning Scene

In [ ]:
# lets add another object to the planning scene and check a waypoint for collisions
waypoint = solution_trajectory.get_waypoint(
    solution_trajectory.num_waypoints - 1
)  # get the second to last waypoint

co2 = CollisionObject()
co2.operation = CollisionObject.ADD
co2.id = "box_2"
co2.header.frame_id = "panda_link0"
co2.primitives = [box_flattened]
pose2 = Pose()
pose2.position.x = 0.15
pose2.position.y = 0.55
pose2.position.z = 0.4
co2.primitive_poses = [pose2]

with psm.read_write() as scene:
    scene.apply_collision_object(co2)
    robot_collision_status = scene.is_state_colliding(
        robot_state=waypoint, group="panda_arm", verbose=False
    )
    print("Waypoint collision status: {}".format(robot_collision_status))
    path_validity_status = scene.is_path_valid(
        path=plan_solution.trajectory, group="panda_arm"
    )
    print("Path validity: {}".format(path_validity_status))

# execute the plan if plan was successful and solution trajectory is not in collision with updated planning scene
if bool(plan_solution) & (not robot_collision_status) & (path_validity_status):
    panda_arm.execute()
else:
    replan_solution = panda_arm.plan()
    if replan_solution:
        panda_arm.execute()

# Return to Ready Pose

In [ ]:
# clear the planning scene
with psm.read_write() as scene:
    scene.remove_all_collision_objects()

# set plan start state to the current state
panda_arm.set_start_state_to_current_state()

# set pose goal using predefined state
panda_arm.set_goal("ready")

# plan to goal
plan_solution = panda_arm.plan()

# execute the plan
if plan_solution:
    panda_arm.execute()

# Playground 